In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        (os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Kaggle kernels usually have torch, torchvision preinstalled.
# But install timm and grad-cam and albumentations if not present.
# Run this cell once (it may take a minute)
!pip install -q timm==0.9.2
!pip install -q pytorch-grad-cam==1.5.2
!pip install -q albumentations==1.3.0
!pip install -q opencv-python-headless


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 48.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 22.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.4 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00:00:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip install pytorch-grad-cam
!pip install monai-weekly==1.4.0
!pip install timm
!pip install pytorch-grad-cam


ERROR: Could not find a version that satisfies the requirement pytorch-grad-cam (from versions: none)
ERROR: No matching distribution found for pytorch-grad-cam
ERROR: Could not find a version that satisfies the requirement monai-weekly==1.4.0 (from versions: 0.4.dev2047, 0.4.dev2048, 0.4.dev2049, 0.4.dev2050, 0.5.dev2051, 0.5.dev2052, 0.5.dev2101, 0.5.dev2102, 0.5.dev2103, 0.5.dev2104, 0.5.dev2105, 0.5.dev2106, 0.5.dev2107, 0.5.dev2108, 0.5.dev2109, 0.5.dev2110, 0.5.dev2111, 0.5.dev2112, 0.5.dev2113, 0.5.dev2114, 0.5.dev2115, 0.6.dev2116, 0.6.dev2117, 0.6.dev2118, 0.6.dev2119, 0.6.dev2120, 0.6.dev2121, 0.6.dev2122, 0.6.dev2123, 0.6.dev2124, 0.6.dev2125, 0.6.dev2126, 0.6.dev2127, 0.7.dev2128, 0.7.dev2129, 0.7.dev2130, 0.7.dev2131, 0.7.dev2132, 0.7.dev2133, 0.7.dev2134, 0.7.dev2135, 0.7.dev2136, 0.7.dev2137, 0.7.dev2138, 0.8.dev2139, 0.8.dev2140, 0.8.dev2141, 0.8.dev2142, 0.8.dev2143, 0.8.dev2144, 0.8.dev2145, 0.8.dev2146, 0.8.dev2147, 0.9.dev2148, 0.9.dev2149, 0.9.dev2150, 0.9.dev2151,

In [8]:
!pip install monai==1.3.0
!pip install nibabel
!pip install pytorch-grad-cam
!pip install timm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.1 MB/s eta 0:00:00a 0:00:01
ERROR: Could not find a version that satisfies the requirement pytorch-grad-cam (from versions: none)
ERROR: No matching distribution found for pytorch-grad-cam


In [9]:
import os
import cv2
import glob
import shutil
import random
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

from monai.networks.nets import SwinUNETR
from monai.transforms import (
    LoadImage,
    EnsureChannelFirst,
    Resize,
    ScaleIntensity,
    ToTensor
)


2025-12-10 17:04:24.244248: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765386264.401550      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765386264.450804      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [7]:
import os, sys, random, math, time, shutil
from pathlib import Path
from glob import glob
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torchvision.transforms as T

import timm
from einops import rearrange

# Grad-CAM
from pytorch_grad_cam import GradCAM, GradCAMPlusPlus
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.model_selection import train_test_split
from skimage import morphology
from skimage.transform import resize as sk_resize

# seed
def seed_everything(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)
seed_everything(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


ModuleNotFoundError: No module named 'pytorch_grad_cam'

In [ ]:
# Kaggle dataset path
DATA_DIR = "/kaggle/input/lung-and-colon-cancer-histopathological-images/lung_colon_image_set/lung_image_sets"
assert os.path.exists(DATA_DIR), f"Dataset folder not found at {DATA_DIR}. Attach dataset to notebook."

# list folders and counts
folders = [d for d in sorted(os.listdir(DATA_DIR)) if os.path.isdir(os.path.join(DATA_DIR,d))]
print("Top-level folders:", folders)
for f in folders:
    p = os.path.join(DATA_DIR,f)
    # count only images
    imgs = len([x for x in os.listdir(p) if x.lower().endswith(('.jpg','.png','.jpeg'))])
    print(f, imgs)


In [ ]:
WORKDIR = "/kaggle/working/lc_lung_threeclass"
os.makedirs(WORKDIR, exist_ok=True)

# detect lung folders
all_dirs = [d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR,d))]
lung_dirs = [d for d in all_dirs if 'lung' in d.lower()]
print("Detected lung folders:", lung_dirs)
if not lung_dirs:
    raise SystemExit("No lung folders detected. Check dataset structure.")

# copy to working dir (symlinks/copy)
for d in lung_dirs:
    src = os.path.join(DATA_DIR, d)
    dst = os.path.join(WORKDIR, d)
    if not os.path.exists(dst):
        shutil.copytree(src, dst)
print("Copied lung folders to:", WORKDIR)


In [ ]:
rows=[]
for cls in sorted(os.listdir(WORKDIR)):
    cls_path = os.path.join(WORKDIR, cls)
    if not os.path.isdir(cls_path): continue
    for f in os.listdir(cls_path):
        if f.lower().endswith(('.png','.jpg','.jpeg')):
            rows.append({"image": os.path.join(cls_path,f), "class": cls})
df = pd.DataFrame(rows)
print("Total images (lung classes):", len(df))
df['label'] = df['class'].astype('category').cat.codes
label_map = dict(enumerate(df['class'].astype('category').cat.categories))
print("Label map:", label_map)
train_df, val_df = train_test_split(df, test_size=0.15, stratify=df['label'], random_state=42)
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
print("Train/Val:", len(train_df), len(val_df))
train_df.head()


In [ ]:
class HistClasDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df.reset_index(drop=True)
        self.transforms = transforms
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = np.array(Image.open(row.image).convert("RGB"))
        if self.transforms:
            augmented = self.transforms(image=img)
            img = augmented['image']
        label = int(row.label)
        return img, label, row.image

train_tf = A.Compose([
    A.Resize(224,224),
    A.RandomRotate90(),
    A.HorizontalFlip(),
    A.VerticalFlip(),
    A.Normalize(),
    ToTensorV2()
])
val_tf = A.Compose([A.Resize(224,224), A.Normalize(), ToTensorV2()])

train_ds = HistClasDataset(train_df, transforms=train_tf)
val_ds = HistClasDataset(val_df, transforms=val_tf)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=2, pin_memory=True)
print("Loaders ready")


In [ ]:
clf = models.resnet18(pretrained=True)
clf.fc = nn.Linear(clf.fc.in_features, len(label_map))
clf = clf.to(device)

optimizer = torch.optim.Adam(clf.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

def train_one_epoch(model, loader):
    model.train()
    tot, acc = 0, 0
    for imgs, labels, _ in loader:
        imgs = imgs.to(device).float()
        labels = labels.to(device)
        optimizer.zero_grad()
        out = model(imgs)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        _, preds = out.max(1)
        tot += labels.size(0)
        acc += (preds==labels).sum().item()
    return acc/tot

def validate(model, loader):
    model.eval()
    tot, acc = 0, 0
    with torch.no_grad():
        for imgs, labels, _ in loader:
            imgs = imgs.to(device).float()
            labels = labels.to(device)
            out = model(imgs)
            _, preds = out.max(1)
            tot += labels.size(0)
            acc += (preds==labels).sum().item()
    return acc/tot

# Train for a few epochs for decent CAMs
EPOCHS_C = 3
for epoch in range(EPOCHS_C):
    tr_acc = train_one_epoch(clf, train_loader)
    va_acc = validate(clf, val_loader)
    print(f"[Classifier] Epoch {epoch+1}/{EPOCHS_C} TrainAcc {tr_acc:.4f} ValAcc {va_acc:.4f}")

# save classifier weights
os.makedirs("/kaggle/working/models", exist_ok=True)
torch.save(clf.state_dict(), "/kaggle/working/models/resnet_clf.pth")
print("Classifier saved at /kaggle/working/models/resnet_clf.pth")


In [ ]:
# Use GradCAM++ targeting last layer
target_layer = clf.layer4[-1]
cam = GradCAMPlusPlus(model=clf, target_layers=[target_layer], use_cuda=torch.cuda.is_available())

PSEUDODIR = "/kaggle/working/pseudo_masks"
os.makedirs(PSEUDODIR, exist_ok=True)

def image_to_float_np(img_path, target_size=(224,224)):
    im = np.array(Image.open(img_path).convert("RGB").resize(target_size))
    im_float = im.astype(np.float32)/255.0
    return im_float

all_images = df['image'].tolist()
print("Creating pseudo masks for", len(all_images), "images. This may take several minutes.")
for idx, img_path in enumerate(all_images):
    img_float = image_to_float_np(img_path, target_size=(224,224))
    # preprocess as train transforms (normalize)
    # Albumentations Normalize uses mean=0.0 std=1.0 by default? we used A.Normalize() which uses imagenet mean/std
    # so replicate that for model forward
    img_norm = (img_float - np.array([0.485,0.456,0.406])) / np.array([0.229,0.224,0.225])
    inp = torch.tensor(np.transpose(img_norm, (2,0,1))[None]).float().to(device)
    with torch.no_grad():
        logits = clf(inp)
        pred = int(logits.argmax(dim=1).cpu().item())
    targets = [ClassifierOutputTarget(pred)]
    gray_cam = cam(input_tensor=inp, targets=targets)[0]  # HxW normalized 0-1 (224x224)
    # resize cam to original image size
    orig = np.array(Image.open(img_path).convert("RGB"))
    cam_resized = sk_resize(gray_cam, (orig.shape[0], orig.shape[1]), preserve_range=True)
    # threshold -> binary mask (heuristic)
    thresh = max(0.25, 0.35 * cam_resized.max())  # safety
    bin_mask = (cam_resized > thresh).astype(np.uint8)
    # morphological cleaning
    bin_mask = morphology.remove_small_objects(bin_mask.astype(bool), min_size=300).astype(np.uint8)
    bin_mask = morphology.binary_closing(bin_mask, morphology.disk(5)).astype(np.uint8)
    mask_path = os.path.join(PSEUDODIR, os.path.basename(img_path))
    Image.fromarray((bin_mask*255).astype(np.uint8)).save(mask_path)
    if (idx+1) % 500 == 0:
        print("Processed:", idx+1)
print("Pseudo masks saved to:", PSEUDODIR)


In [ ]:
seg_rows=[]
for row in df.itertuples():
    img = row.image
    mask = os.path.join(PSEUDODIR, os.path.basename(img))
    if os.path.exists(mask):
        seg_rows.append({"image": img, "mask": mask, "label": row.label})
seg_df = pd.DataFrame(seg_rows)
print("Seg pairs found:", len(seg_df))
if len(seg_df)==0:
    raise SystemExit("No pseudo masks found. Check PSEUDODIR.")

seg_train, seg_val = train_test_split(seg_df, test_size=0.15, stratify=seg_df['label'], random_state=42)
seg_train = seg_train.reset_index(drop=True)
seg_val = seg_val.reset_index(drop=True)
print("Seg Train/Val:", len(seg_train), len(seg_val))


In [ ]:
class SegDataset(Dataset):
    def __init__(self, df, transforms=None, img_size=384):
        self.df = df.reset_index(drop=True)
        self.transforms = transforms
        self.img_size = img_size
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = np.array(Image.open(row.image).convert("RGB"))
        mask = np.array(Image.open(row.mask).convert("L"))
        mask = (mask > 127).astype(np.uint8)
        if self.transforms:
            augmented = self.transforms(image=img, mask=mask)
            img = augmented['image']
            mask = augmented['mask']
        # ensure mask shape [H,W] then add channel
        mask = torch.tensor(mask, dtype=torch.float32).unsqueeze(0)
        return img.float(), mask.float()

train_tf_seg = A.Compose([
    A.Resize(384,384),
    A.RandomRotate90(),
    A.HorizontalFlip(),
    A.VerticalFlip(),
    A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
    ToTensorV2()
])
val_tf_seg = A.Compose([
    A.Resize(384,384),
    A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
    ToTensorV2()
])

train_ds_seg = SegDataset(seg_train, transforms=train_tf_seg, img_size=384)
val_ds_seg = SegDataset(seg_val, transforms=val_tf_seg, img_size=384)
train_loader_seg = DataLoader(train_ds_seg, batch_size=8, shuffle=True, num_workers=2, pin_memory=True)
val_loader_seg = DataLoader(val_ds_seg, batch_size=8, shuffle=False, num_workers=2, pin_memory=True)
print("Seg dataloaders ready. Train batches:", len(train_loader_seg), "Val batches:", len(val_loader_seg))


In [ ]:
# We'll use timm swin model features_only to get multi-scale features
class PatchEmbedSwin(nn.Module):
    def __init__(self, model_name='swin_base_patch4_window7_224', pretrained=True):
        super().__init__()
        self.swin = timm.create_model(model_name, pretrained=pretrained, features_only=True, out_indices=(0,1,2,3))
    def forward(self, x):
        return self.swin(x)  # list of feature maps [f1,f2,f3,f4]

class DecoderBlock(nn.Module):
    def __init__(self, in_ch, skip_ch, out_ch):
        super().__init__()
        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch + skip_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )
    def forward(self, x, skip):
        x = self.up(x)
        if x.shape[-2:] != skip.shape[-2:]:
            x = F.interpolate(x, size=skip.shape[-2:], mode='bilinear', align_corners=False)
        x = torch.cat([x, skip], dim=1)
        return self.conv(x)

class SwinUNetR(nn.Module):
    def __init__(self, n_classes=1, swin_name='swin_base_patch4_window7_224', pretrained=True):
        super().__init__()
        self.encoder = PatchEmbedSwin(model_name=swin_name, pretrained=pretrained)
        # dummy forward to find channels (use small tensor)
        with torch.no_grad():
            dummy = torch.randn(1,3,384,384)
            feats = self.encoder(dummy)
            chs = [f.shape[1] for f in feats]  # [c1,c2,c3,c4]
        # decoder blocks
        self.decoder4 = DecoderBlock(in_ch=chs[-1], skip_ch=chs[-2], out_ch=512)
        self.decoder3 = DecoderBlock(in_ch=512, skip_ch=chs[-3], out_ch=256)
        self.decoder2 = DecoderBlock(in_ch=256, skip_ch=chs[-4], out_ch=128)
        self.decoder1 = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False),
            nn.Conv2d(128, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )
        self.head = nn.Conv2d(64, n_classes, kernel_size=1)
    def forward(self, x):
        feats = self.encoder(x)  # [f1,f2,f3,f4]
        f1,f2,f3,f4 = feats
        x = self.decoder4(f4, f3)
        x = self.decoder3(x, f2)
        x = self.decoder2(x, f1)
        x = self.decoder1(x)
        out = self.head(x)  # raw logits (no sigmoid)
        return out

# instantiate
model = SwinUNetR(n_classes=1, swin_name='swin_base_patch4_window7_224', pretrained=True)
model = model.to(device)
print("Model ready. Param count:", sum(p.numel() for p in model.parameters())/1e6, "M")


In [ ]:
# BCEWithLogits + Dice (use logits directly)
bce_logits = nn.BCEWithLogitsLoss()

def dice_loss_from_logits(logits, target, smooth=1e-6):
    pred = torch.sigmoid(logits)
    pred = pred.contiguous()
    target = target.contiguous()
    intersection = (pred * target).sum(dim=(2,3))
    denom = pred.sum(dim=(2,3)) + target.sum(dim=(2,3))
    loss = 1 - ((2. * intersection + smooth) / (denom + smooth))
    return loss.mean()

def compute_iou_from_logits(logits, target, thr=0.5):
    pred = torch.sigmoid(logits)
    pred_bin = (pred > thr).float()
    inter = (pred_bin * target).sum(dim=(2,3))
    union = (pred_bin + target).clamp(0,1).sum(dim=(2,3))
    iou = (inter + 1e-6) / (union + 1e-6)
    return iou.mean().item()

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.5, verbose=True)


In [ ]:
num_epochs = 12   # adjust as compute budget allows
best_val_iou = 0.0
os.makedirs("/kaggle/working/seg_models", exist_ok=True)

for epoch in range(num_epochs):
    model.train()
    train_loss_sum = 0.0
    train_iou_sum = 0.0
    nb = 0
    for imgs, masks in train_loader_seg:
        imgs = imgs.to(device).float()
        masks = masks.to(device).float()
        preds = model(imgs)  # logits [B,1,H,W]
        # ensure same size
        if preds.shape != masks.shape:
            preds = F.interpolate(preds, size=masks.shape[-2:], mode='bilinear', align_corners=False)
        loss_bce = bce_logits(preds, masks)
        loss_dice = dice_loss_from_logits(preds, masks)
        loss = loss_bce + loss_dice
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss_sum += loss.item()
        train_iou_sum += compute_iou_from_logits(preds.detach().cpu(), masks.detach().cpu())
        nb += 1
    train_loss = train_loss_sum / max(nb,1)
    train_iou = train_iou_sum / max(nb,1)

    # validation
    model.eval()
    val_loss_sum = 0.0
    val_iou_sum = 0.0
    nb = 0
    with torch.no_grad():
        for imgs, masks in val_loader_seg:
            imgs = imgs.to(device).float()
            masks = masks.to(device).float()
            preds = model(imgs)
            if preds.shape != masks.shape:
                preds = F.interpolate(preds, size=masks.shape[-2:], mode='bilinear', align_corners=False)
            loss_bce = bce_logits(preds, masks)
            loss_dice = dice_loss_from_logits(preds, masks)
            loss = loss_bce + loss_dice
            val_loss_sum += loss.item()
            val_iou_sum += compute_iou_from_logits(preds.cpu(), masks.cpu())
            nb += 1
    val_loss = val_loss_sum / max(nb,1)
    val_iou = val_iou_sum / max(nb,1)
    scheduler.step(val_loss)

    print(f"Epoch {epoch+1}/{num_epochs} TrainLoss:{train_loss:.4f} TrainIoU:{train_iou:.4f} ValLoss:{val_loss:.4f} ValIoU:{val_iou:.4f}")

    # save best
    if val_iou > best_val_iou:
        best_val_iou = val_iou
        torch.save(model.state_dict(), f"/kaggle/working/seg_models/swinunetr_best.pth")
        print("Saved best model at epoch", epoch+1, "ValIoU:", best_val_iou)


In [ ]:
# show some validation images with predicted overlay and pseudo GT
model.eval()
with torch.no_grad():
    imgs, masks = next(iter(val_loader_seg))
    imgs = imgs.to(device).float()
    preds = model(imgs)
    preds = torch.sigmoid(preds).cpu().numpy()
    imgs = imgs.cpu().numpy()
    masks = masks.cpu().numpy()
    n_show = min(6, imgs.shape[0])
    plt.figure(figsize=(12,4*n_show//3 + 2))
    for i in range(n_show):
        img = np.transpose(imgs[i], (1,2,0))
        # unnormalize (we used imagenet norm)
        mean = np.array([0.485,0.456,0.406]); std = np.array([0.229,0.224,0.225])
        img = (img * std) + mean
        img = np.clip(img, 0, 1)
        pred_mask = preds[i,0]
        gt_mask = masks[i,0]
        plt.subplot(n_show,3,3*i+1); plt.imshow(img); plt.axis('off'); plt.title('Image')
        plt.subplot(n_show,3,3*i+2); plt.imshow(img); plt.imshow(pred_mask, alpha=0.45); plt.axis('off'); plt.title('Pred overlay')
        plt.subplot(n_show,3,3*i+3); plt.imshow(gt_mask, cmap='gray'); plt.axis('off'); plt.title('Pseudo GT')
    plt.tight_layout()
    plt.show()


In [ ]:
# Save final model (already saved best)
MODEL_PATH = "/kaggle/working/seg_models/swinunetr_best.pth"
print("Best model path:", MODEL_PATH)
# Inference helper: load and predict on a single image
def load_model(path=MODEL_PATH):
    m = SwinUNetR(n_classes=1, swin_name='swin_base_patch4_window7_224', pretrained=False)
    m.load_state_dict(torch.load(path, map_location='cpu'))
    m.to(device).eval()
    return m

def predict_image(model, img_path, out_size=(384,384), thr=0.5):
    img = np.array(Image.open(img_path).convert("RGB"))
    img_res = sk_resize(img, out_size, preserve_range=True).astype(np.uint8)
    img_norm = (img_res.astype(np.float32)/255.0 - np.array([0.485,0.456,0.406])) / np.array([0.229,0.224,0.225])
    inp = torch.tensor(np.transpose(img_norm, (2,0,1))[None]).float().to(device)
    with torch.no_grad():
        logits = model(inp)
        prob = torch.sigmoid(logits)[0,0].cpu().numpy()
    mask = (prob > thr).astype(np.uint8)
    return img_res, prob, mask

# Example usage:
# model_loaded = load_model()
# img_res, prob_map, mask_pred = predict_image(model_loaded, seg_val.iloc[0].image)
